# 4-7. 프로젝트: 멋진 작사가 만들기

## Step 1. 데이터 다운로드

이미 실습(1) 데이터 다듬기에서 Cloud shell에

심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면,

~/aiffel/lyricist/data/lyrics에 데이터가 있습니다.

## Step 2. 데이터 읽어오기

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요.

glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [23]:
# 필요한 모듈 import 하기 

import re    
import glob
import numpy as np         
import tensorflow as tf    
import os

In [25]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:9])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face [Verse 1]']


In [26]:
# 공백인 문장은 길이를 검사하여 길이가 0이라면 제외를 시켜 봅니다.

for idx, sentence in enumerate(raw_corpus): # enumerate 열거하다 라는 뜻으로, 리스트가 있는 경우 순서와 리스트의 값을 전달하는 기능
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Hook]
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face [Verse 1]
Somethin' ain't right when we talkin'


## Step 3. 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠?

이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다.

너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.

그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을

학습 데이터에서 제외하기 를 권합니다.

### 문장 전처리

#### 1. 입력된 문장을 정규 표현식을 이용해서 정제합니다.

In [27]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


#### 2. preprocess_sentence로 정제한 문장을 모음

In [28]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> hook <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face verse <end>',
 '<start> somethin ain t right when we talkin <end>']

## Step 4. 평가 데이터셋 분리

훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후,

sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다.

단어장의 크기는 12,000 이상 으로 설정하세요!

총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

#### 3. tokenize() 함수로 데이터를 Tensor로 변환 (Tokenizer와 pad_sequences를 사용)

단어별로 토근화 할 필요가 있습니다.

컴퓨터는 문장을 문장으로보지않고 숫자로 봐야하기 때문에 토큰화 라는것을 해야하는데

토큰화란, 단어 하나에 숫자를 매칭시켜주는 개념입니다.

 ex) I = 0 , love = 1, you = 2

In [29]:
# tf 의 tokenize() 함수를 사용해서 12000개 단어 개수를 숫자로 바꿔줍니다.

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        # 전체 단어의 수
        num_words=12000, 
        # 이 함수에서 제공하는 문장 전처리 (but, 이미 앞에서 진행했으므로 사용 X)
        filters=' ',
        # 12000 단어에 속하지 않으면, unk 로 바꿔줍니다.
        oov_token="<unk>"
    )
    # corpus로 tokenizer 내부의 단어장을 완성합니다. (= 데이터 구축)
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다.
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 문장의 길이를 맞추고 숫자로 반환하기 위해 작업을 합니다.
    # 문장의 길이는 가장 긴 문장을 기준으로 pandding합니다.
    # 여기서 maxien을 설정해 주었는데 이상치에 대응하기 할 수 있다고 합니다.
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다.
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15) # maxlen 은 여기에서 어떤 것을 의미할까?  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [30]:
tensor, tokenizer = tokenize(corpus)

tensor.shape

[[   2  953    3 ...    0    0    0]
 [   2    5   91 ...   10   12    3]
 [   2   45  134 ...    0    0    0]
 ...
 [   2  211    3 ...    0    0    0]
 [   2  399    9 ...    0    0    0]
 [   2    9 1294 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc163362d90>


(175749, 15)

In [31]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력

print(tensor[:3, :10])

[[  2 953   3   0   0   0   0   0   0   0]
 [  2   5  91 106  60  30 161   4  11 134]
 [  2  45 134  29  10  12   3   0   0   0]]


In [32]:
# tokenizer에 구축된 단어 사전의 인덱스

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
        
print(tokenizer.num_words)

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
12000


#### 4. sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리

In [33]:
from sklearn.model_selection import train_test_split

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2 953   3   0   0   0   0   0   0   0   0   0   0   0]
[953   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [39]:
# 사이킷 런 패키지를 이용해서 위에서 train과 target을 train과 validation 셋으로 분리를 해줍니다. 8:2로 분리를 하겠습니다.

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42)

In [40]:
# train의 맨 뒷 단어를 뺀 문장을 분석해서 앞 단어가 없는 target을 찾는것이 목적이기 때문에 shape는 동일한 모습을 하고있습니다.

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [ ]:
# 아래 코드를 진행하는 이유를 모르겠음...ㅠ

In [43]:
BUFFER_SIZE = len(src_input) # 텐서의 1차원, 전체 문장의 개수
BATCH_SIZE = 256 # 문장의 개수
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에

val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!

(Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

### 1. 모델 구조 정의

In [44]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size): 
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

- 하이퍼 파라미터 지정과 모델 생성
    - embedding_size를 256으로 올려 고려하는 feature의 수를 늘림
    - hidden_size를 2048로 올려 각 레이어의 노드 수를 늘림

### 2. 모델 생성

In [46]:
embedding_size = 256 # 단어 하나의 특징 수
hidden_size = 2048 # 퍼셉트론의 갯수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

- 생성된 모델 살펴보기: model_summary
    - 하나의 배치만 이용하면 임시 모델을 생성해볼 수 있다

In [47]:
for src_sample, tgt_sample in dataset.take(1): break

In [48]:
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.1900426e-04,  4.1354677e-05,  6.8362817e-05, ...,
         -4.7425750e-05, -1.9203319e-04,  1.5761470e-04],
        [-1.8513683e-04, -3.0734524e-04,  9.5235691e-07, ...,
         -1.0465698e-04, -7.5803131e-05, -4.8671277e-06],
        [-4.0016428e-04, -4.4669647e-04, -2.3051373e-04, ...,
         -2.0140865e-04,  4.5769685e-04,  1.0092894e-05],
        ...,
        [ 1.8067231e-03, -7.2623603e-04,  1.0453196e-03, ...,
          1.6668050e-03,  7.1167841e-04,  9.0292125e-04],
        [ 2.2726848e-03, -9.7713806e-04,  1.4572965e-03, ...,
          2.0703541e-03,  6.4369617e-04,  1.0825038e-03],
        [ 2.6888498e-03, -1.2192833e-03,  1.8530354e-03, ...,
          2.3999866e-03,  5.7702389e-04,  1.2508555e-03]],

       [[-1.1900426e-04,  4.1354677e-05,  6.8362817e-05, ...,
         -4.7425750e-05, -1.9203319e-04,  1.5761470e-04],
        [-5.2945327e-04,  5.2637730e-05,  1.8553706e-04, ...,
          4.6176017e-05, 

In [49]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


- optimizer와 loss function 지정
    - optimizer: SGD를 사용해보았으나 Adam이 더 성능이 좋음

In [59]:
history = []
epochs = 10

In [50]:
optimizer = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.SGD()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [51]:
model.compile(loss=loss, optimizer=optimizer)

- checkpoint 설정: ModelCheckpoint

In [52]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### 3. 모델 학습: fit

In [53]:
model.fit(x = enc_train, y = dec_train, epochs=10, batch_size=128, validation_data = (enc_val_val, dec_val_val)
         , callbacks =[checkpoint_callback])

Epoch 1/10
1099/1099 [==============================] - 517s 468ms/step - loss: 3.8748 - val_loss: 3.0033
Epoch 2/10
1099/1099 [==============================] - 516s 470ms/step - loss: 2.9197 - val_loss: 2.6091
Epoch 3/10
1099/1099 [==============================] - 514s 468ms/step - loss: 2.5354 - val_loss: 2.2583
Epoch 4/10
1099/1099 [==============================] - 515s 469ms/step - loss: 2.1670 - val_loss: 1.9492
Epoch 5/10
1099/1099 [==============================] - 514s 468ms/step - loss: 1.8273 - val_loss: 1.7017
Epoch 6/10
1099/1099 [==============================] - 513s 467ms/step - loss: 1.5364 - val_loss: 1.5098
Epoch 7/10
1099/1099 [==============================] - 515s 469ms/step - loss: 1.3132 - val_loss: 1.3744
Epoch 8/10
1099/1099 [==============================] - 516s 469ms/step - loss: 1.1450 - val_loss: 1.2884
Epoch 9/10
1099/1099 [==============================] - 514s 468ms/step - loss: 1.0358 - val_loss: 1.2411
Epoch 10/10
1099/1099 [=======================


### 노드 커멘트

데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

# 하지만... 한국인으로서 참기 너무 어려웠습니다 ㅠㅠ

### 4. 실습 : 잘 만들어졌는지 평가하기

In [55]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [57]:
# 위 문장 생성 함수를 실행

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love ma little nasty girl <end> '

In [60]:
import matplotlib.pyplot as plt
def plot_curve(epochs, hist, list_of_metrics):
    
    fig, ax = plt.subplots(1,2,figsize = (12, 8))
    
    for i in range(len(ax)):
        ax[i].set_xlabel('Epochs')
        ax[i].set_ylabel('Value')
        
        for n in range(len(list_of_metrics)):
            if i == 0:
                y = hist[list_of_metrics[n]]
                if n == 0:
                    ax[i].plot(epochs, y, label="train")
                else:
                    ax[i].plot(epochs, y, label="val")
                ax[i].set_title('Loss')
                ax[i].legend(loc='upper right')
                if n == 1:
                    break
            else:
                if n >= 2:
                    y = hist[list_of_metrics[n]]
                    if n == 2:
                        ax[i].plot(epochs, y, label="train")
                    else:
                        ax[i].plot(epochs, y, label="val")
                    ax[i].set_title('Accuracy')
                    ax[i].legend(loc='lower right')
                    
    plt.show()
plot_curve(history.epoch, history.history, ['loss', 'val_loss'])

AttributeError: 'list' object has no attribute 'epoch'

그래프를 보니 train은 학습하면 할 수록 과적합이 되서 그런지

점점 loss가 줄어드는 모습을 보입니다.

그리고 validation에서의 loss는 train의 학습으로는 한계가 있는지

점점 loss가 줄어드는 폭이 좁아지는 것을 볼 수 있었습니다.

더 좋은 결과를 얻기 위해 어떤 과정이 남아 있을까?

lstm의 하이퍼 파라미터를 수정 또는 층을 늘린다.

epochs에 earlystopping을 추가하여 가장 강력할 때 멈춘다.

각 lstm 층마다 과적합 방지 기법을 사용한다. ( drop out, batchnormalization)

cross validation을 사용한다.

optimizer를 Adam말고 시퀀스 데이터에 더 적합하다고 알려진 rmsprop를 사용한다.